# Netflix movie search

Finding out what to watch is tough choice mainly because of the abundance of options and the scarcity of time.
Let's follow a data-driven approach to arrive at a decision!

In our [dataset](https://www.kaggle.com/datasets/dgoenrique/netflix-movies-and-tv-shows) of movies, we know the
- description,
- genre
- title
- and the release_year
of the movie.

Putting together a vector index on top of these embedded inputs will create a space where we can search semantically to find our movie choice for tonight.

We are going to browse the movies

- searching with an idea (heartfelt romantic comedy)
- tweak around the results giving more importance to matches in certain input fields
- search in description, genre and title with different search terms for each
- and after finding a close enough movie (that is not quite it), search around utilizing that one, too

## Boilerplate

### Installation

In [20]:
%pip install superlinked==36.1.0

### Imports and constants

In [21]:
import ast
from datetime import timedelta, datetime
import os
import sys

import altair as alt
import pandas as pd
from superlinked import framework as sl

alt.renderers.enable(sl.get_altair_renderer())
alt.data_transformers.disable_max_rows()
pd.set_option("display.max_colwidth", 190)

In [22]:
YEAR_IN_DAYS = 365
TOP_N = 10
DATASET_URL = "https://storage.googleapis.com/superlinked-notebook-netflix-shows-dataset/titles.csv"
# as our latest movie is from 2022 we can set this as the time of the analysis
END_OF_2022_TS = int(datetime(2022, 12, 31, 23, 59).timestamp())
EXECUTOR_DATA = {sl.CONTEXT_COMMON: {sl.CONTEXT_COMMON_NOW: END_OF_2022_TS}}

## Prepare dataset

In [23]:
NROWS = int(os.getenv("NOTEBOOK_TEST_ROW_LIMIT", str(sys.maxsize)))
movie_df = pd.read_csv(DATASET_URL, nrows=NROWS)
# keep only relevant columns
movie_df = (
    pd.DataFrame(movie_df[["description", "genres", "title", "release_year", "id"]])
    .drop_duplicates(subset=["description"])
    .dropna(how="any")
)
# join genres to create a textual column
movie_df["genres"] = movie_df["genres"].apply(lambda x: " ".join(ast.literal_eval(x)))
# convert release year to a timestamp - always the 1st of January
movie_df["timestamp"] = [
    int(pd.Timestamp(year=year, month=1, day=1).timestamp()) for year in movie_df["release_year"].tolist()
]
movie_df.head()

,description,genres,title,release_year,id,timestamp
0,"This collection includes 12 World War II-era propaganda films — many of which are graphic and offensive — discussed in the docuseries ""Five Came Back.""",documentation,Five Came Back: The Reference Films,1945,ts300399,-788918400
1,A mentally unstable Vietnam War veteran works as a night-time taxi driver in New York City where the perceived decadence and sleaze feed his urge for violent action.,drama crime,Taxi Driver,1976,tm84618,189302400
2,"Intent on seeing the Cahulawassee River before it's turned into one huge lake, outdoor fanatic Lewis Medlock takes his friends on a river-rafting trip they'll never forget into the dange...",drama action thriller european,Deliverance,1972,tm154986,63072000
3,"King Arthur, accompanied by his squire, recruits his Knights of the Round Table, including Sir Bedevere the Wise, Sir Lancelot the Brave, Sir Robin the Not-Quite-So-Brave-As-Sir-Lancelot...",fantasy action comedy,Monty Python and the Holy Grail,1975,tm127384,157766400
4,"12 American military prisoners in World War II are ordered to infiltrate a well-guarded enemy château and kill the Nazi officers vacationing there. The soldiers, most of whom are facing ...",war action,The Dirty Dozen,1967,tm120801,-94694400


In [24]:
# let's use 4, 10 and 40 years as our period times so that populous areas get more focus
alt.Chart(pd.DataFrame(movie_df[["release_year"]].astype(str))).mark_bar().encode(x=alt.X("release_year"), y="count()")

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


## Set up Superlinked

In [25]:
# accommodate our inputs in a typed schema
class MovieSchema(sl.Schema):
    description: sl.String
    title: sl.String
    release_timestamp: sl.Timestamp
    genres: sl.String
    id: sl.IdField

In [26]:
movie = MovieSchema()

In [27]:
# textual fields are embedded using a sentence-transformers model
description_space = sl.TextSimilaritySpace(
    text=movie.description, model="sentence-transformers/paraphrase-MiniLM-L3-v2"
)
title_space = sl.TextSimilaritySpace(text=movie.title, model="sentence-transformers/paraphrase-MiniLM-L3-v2")
genre_space = sl.TextSimilaritySpace(text=movie.genres, model="sentence-transformers/paraphrase-MiniLM-L3-v2")
# release date are encoded using our recency space
# periodtimes aim to reflect notable breaks in our scores
recency_space = sl.RecencySpace(
    timestamp=movie.release_timestamp,
    period_time_list=[
        sl.PeriodTime(timedelta(days=4 * YEAR_IN_DAYS)),
        sl.PeriodTime(timedelta(days=10 * YEAR_IN_DAYS)),
        sl.PeriodTime(timedelta(days=40 * YEAR_IN_DAYS)),
    ],
    negative_filter=-0.25,
)

In [28]:
movie_index = sl.Index(spaces=[description_space, title_space, genre_space, recency_space])

In [29]:
query_text_param = sl.Param("query_text")

simple_query = (
    sl.Query(
        movie_index,
        weights={
            description_space: sl.Param("description_weight"),
            title_space: sl.Param("title_weight"),
            genre_space: sl.Param("genre_weight"),
            recency_space: sl.Param("recency_weight"),
        },
    )
    .find(movie)
    .similar(description_space, query_text_param)
    .similar(title_space, query_text_param)
    .similar(genre_space, query_text_param)
    .select([movie.description, movie.title, movie.genres, movie.release_timestamp])
    .limit(sl.Param("limit"))
)

advanced_query = (
    sl.Query(
        movie_index,
        weights={
            description_space: sl.Param("description_weight"),
            title_space: sl.Param("title_weight"),
            genre_space: sl.Param("genre_weight"),
            recency_space: sl.Param("recency_weight"),
        },
    )
    .find(movie)
    .similar(description_space, sl.Param("description_query_text"))
    .similar(title_space, sl.Param("title_query_text"))
    .similar(genre_space, sl.Param("genre_query_text"))
    .select([movie.description, movie.title, movie.genres, movie.release_timestamp])
    .limit(sl.Param("limit"))
)

In [30]:
df_parser = sl.DataFrameParser(schema=movie, mapping={movie.release_timestamp: "timestamp"})

In [31]:
source: sl.InMemorySource = sl.InMemorySource(movie, parser=df_parser)
executor: sl.InMemoryExecutor = sl.InMemoryExecutor(sources=[source], indices=[movie_index], context_data=EXECUTOR_DATA)
app: sl.InMemoryApp = executor.run()

In [32]:
source.put([movie_df])

## Understanding recency

Recency can seem quite complex at first, let's see how the score looks like for the relevant time periods

In [33]:
recency_plotter = sl.RecencyPlotter(recency_space, context_data=EXECUTOR_DATA)
chart = recency_plotter.plot_recency_curve()
chart

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


notice the breaks in the score at 4, 10 and 40 years - those are our period times. Titles older than 40 years get `negative_filter` score.

## Run queries

### Search with user queries

With the simple query, I can search with my text in all of the fields

In [34]:
result = app.query(
    simple_query,
    query_text="Heartfelt romantic comedy",
    description_weight=1,
    title_weight=1,
    genre_weight=1,
    recency_weight=0,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,"When a group of people meets at the same party, they form four different relationships, each experiencing similar phases of love.","Love, Surreal and Odd",comedy romance,tm313118,0.600616,2017
1,"An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted roman...",Love Hard,romance comedy,tm1093099,0.563954,2021
2,Romantic anthology web series revolving around 12 stories of 12 months of young people on their paths to love.,Love Daily,drama romance,ts90869,0.559345,2018
3,"In this romantic comedy, several friends, each dealing with unhappy love lives, turn to each other for help - but not always with the best results.",F*ck Love Too,comedy romance,tm1203307,0.558514,2022
4,Aspiring pop star Erica ends up as the entertainment at her ex-fiancé’s wedding after reluctantly taking a gig at a luxurious island resort while in the wake of a music career meltdown.,Resort to Love,romance comedy,tm1037704,0.557839,2021
5,"'Love Actually' follows the lives of eight very different couples dealing with their love lives, in various loosely and interrelated tales, all set during a frantic month before Christma...",Love Actually,drama comedy romance,tm133767,0.551500,2003
6,Exploration into the tense relationship of success and romance.,Love or Money,romance,tm997617,0.551167,2021
7,"It tells the love story of two childhood sweethearts that spans 19 years. Chen Xiao Xi is a cute and small girl with a lot of positive energy. She gets to know Jiang Chen, a tall and pro...",A Love So Beautiful,comedy romance,ts77034,0.549566,2017
8,Rebellious Mickey and good-natured Gus navigate the thrills and agonies of modern relationships.,Love,comedy drama romance,ts38511,0.538660,2016
9,"A young woman develops romantic feelings for her best friend, but problems arise when another gal enters the picture.",Must Be... Love,comedy romance,tm188290,0.535215,2013


After looking at the results, I see some titles I have already seen. I can bias towards recent titles by upweighting recency. Weights are normalized to have unit sum, so you don't have to worry about how you set them.

In [35]:
second_result = app.query(
    simple_query,
    query_text="Heartfelt romantic comedy",
    description_weight=1,
    title_weight=1,
    genre_weight=1,
    recency_weight=3,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,"When a group of people meets at the same party, they form four different relationships, each experiencing similar phases of love.","Love, Surreal and Odd",comedy romance,tm313118,0.600616,2017
1,"An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted roman...",Love Hard,romance comedy,tm1093099,0.563954,2021
2,Romantic anthology web series revolving around 12 stories of 12 months of young people on their paths to love.,Love Daily,drama romance,ts90869,0.559345,2018
3,"In this romantic comedy, several friends, each dealing with unhappy love lives, turn to each other for help - but not always with the best results.",F*ck Love Too,comedy romance,tm1203307,0.558514,2022
4,Aspiring pop star Erica ends up as the entertainment at her ex-fiancé’s wedding after reluctantly taking a gig at a luxurious island resort while in the wake of a music career meltdown.,Resort to Love,romance comedy,tm1037704,0.557839,2021
5,"'Love Actually' follows the lives of eight very different couples dealing with their love lives, in various loosely and interrelated tales, all set during a frantic month before Christma...",Love Actually,drama comedy romance,tm133767,0.551500,2003
6,Exploration into the tense relationship of success and romance.,Love or Money,romance,tm997617,0.551167,2021
7,"It tells the love story of two childhood sweethearts that spans 19 years. Chen Xiao Xi is a cute and small girl with a lot of positive energy. She gets to know Jiang Chen, a tall and pro...",A Love So Beautiful,comedy romance,ts77034,0.549566,2017
8,Rebellious Mickey and good-natured Gus navigate the thrills and agonies of modern relationships.,Love,comedy drama romance,ts38511,0.538660,2016
9,"A young woman develops romantic feelings for her best friend, but problems arise when another gal enters the picture.",Must Be... Love,comedy romance,tm188290,0.535215,2013


Still using the simple query, I can give more weight to spaces if I think my query is more related to that space - matches there should count more. Here I give additional weight to the genre, leave the description as is, and downweight the title as my query text is mostly a genre with some additional context. I keep recency with unit weight too, as I would like my results to be a bit biased towards recent movies.

In [36]:
result = app.query(
    simple_query,
    query_text="Heartfelt romantic comedy",
    description_weight=1,
    title_weight=0.1,
    genre_weight=2,
    recency_weight=1,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,"In this romantic comedy, several friends, each dealing with unhappy love lives, turn to each other for help - but not always with the best results.",F*ck Love Too,comedy romance,tm1203307,0.641047,2022
1,"An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted roman...",Love Hard,romance comedy,tm1093099,0.608691,2021
2,"A relatable romance drama about a couple in their 30s preparing for marriage. While they were expecting a happy ending like something out of a fairy tale, the reality of their preparatio...",Welcome to Wedding Hell,drama romance,ts344043,0.596496,2022
3,This black humor pan-Arabic anthology series is about love in general – and relationships in particular.,"Love, Life & Everything in Between",comedy drama romance,ts329711,0.593643,2022
4,Short films follow young adults as they navigate the gamut of emotions that come with finding romantic connection in unexpected places.,Feels Like Ishq,drama romance comedy,ts283610,0.592501,2021
5,Incurable romantic Lotte's life is upended when her plans for a picture-perfect wedding unravel--just as her self-absorbed sister gets engaged.,Just Say Yes,comedy romance,tm904862,0.590069,2021
6,"In pursuit of both success and validation, a group of tech-savvy individuals juggle intimate encounters, first impressions and romantic opportunities.",Slay,romance comedy,tm1038025,0.589858,2021
7,"In this rom-com challenging the concept of soulmates, parallel storylines portray four single friends as they pair up in different couple combinations.",Four to Dinner,comedy romance,tm1137862,0.584342,2022
8,Aspiring pop star Erica ends up as the entertainment at her ex-fiancé’s wedding after reluctantly taking a gig at a luxurious island resort while in the wake of a music career meltdown.,Resort to Love,romance comedy,tm1037704,0.583462,2021
9,"Guille decides it's time to take the next step and that's how she looks to marry her sweetheart, but things get complicated and nothing goes as planned.","Let's Tie the Knot, Honey!",comedy romance,tm1204412,0.581410,2022


With the advanced query, I can even supply different search terms for each attribute of the movie.

In [37]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=1,
    title_weight=1,
    genre_weight=1,
    recency_weight=0,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,"'Love Actually' follows the lives of eight very different couples dealing with their love lives, in various loosely and interrelated tales, all set during a frantic month before Christma...",Love Actually,drama comedy romance,tm133767,0.722857,2003
1,Rebellious Mickey and good-natured Gus navigate the thrills and agonies of modern relationships.,Love,comedy drama romance,ts38511,0.722234,2016
2,"An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted roman...",Love Hard,romance comedy,tm1093099,0.701640,2021
3,A rising black painter tries to break into a competitive art world while balancing an unexpected romance with an ambitious law student.,Really Love,romance drama,tm946360,0.684617,2020
4,Romantic anthology web series revolving around 12 stories of 12 months of young people on their paths to love.,Love Daily,drama romance,ts90869,0.670919,2018
5,"Adam and Marklin’s 5-year relationship has gone from a passionate flame to a simmer, forcing them to reconcile with each other’s shortcomings while their friends endlessly search for lov...",Almost Love,comedy drama romance,tm472195,0.667730,2019
6,"Love is as tough as it is sweet for a lovestruck teenager, whose relationship with her next-door neighbor transforms as they grow into adulthood.",A Love So Beautiful,comedy drama,ts357033,0.641233,2020
7,"Near by Christmas, in an old and charming town in Transylvania, Sebastian and Aprilia start a beautiful love story. However, nothing is as simple as it would seem. Being influenced by th...",Love Is a Story,romance european comedy,tm368725,0.638091,2015
8,This black humor pan-Arabic anthology series is about love in general – and relationships in particular.,"Love, Life & Everything in Between",comedy drama romance,ts329711,0.631610,2022
9,Laida Magtalas is a modern-day Belle who works hard to provide for her family while hoping that someday she will meet her prince charming and that they will live happily ever after toget...,A Very Special Love,comedy drama romance,tm28024,0.630495,2008


And even give different weights to each subsearch if I really do care that the title is related to love but I am not really emotionally attached to my description of the movie I would want to see.

In [38]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=0.2,
    title_weight=3,
    genre_weight=1,
    recency_weight=0,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,Rebellious Mickey and good-natured Gus navigate the thrills and agonies of modern relationships.,Love,comedy drama romance,ts38511,0.724493,2016
1,The story of a family and the various situations navigated by a husband and wife.,Love,thriller drama,tm946277,0.676508,2020
2,"'Love Actually' follows the lives of eight very different couples dealing with their love lives, in various loosely and interrelated tales, all set during a frantic month before Christma...",Love Actually,drama comedy romance,tm133767,0.645147,2003
3,A rising black painter tries to break into a competitive art world while balancing an unexpected romance with an ambitious law student.,Really Love,romance drama,tm946360,0.640506,2020
4,"Adam and Marklin’s 5-year relationship has gone from a passionate flame to a simmer, forcing them to reconcile with each other’s shortcomings while their friends endlessly search for lov...",Almost Love,comedy drama romance,tm472195,0.617771,2019
5,"The story of Richard and Mildred Loving, an interracial couple, whose challenge of their anti-miscegenation arrest for their marriage in Virginia led to a legal battle that would end at ...",Loving,drama romance,tm219233,0.609814,2016
6,"An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted roman...",Love Hard,romance comedy,tm1093099,0.609514,2021
7,"In order to receive a bone marrow transplant quicker and be able to continue her career as an actress, Xia Lin enters into a secret marriage with Ling Yi Zhou, the CEO of a company. Desp...",Well-Intended Love,drama romance,ts88133,0.581787,2019
8,"It tells the love story of two childhood sweethearts that spans 19 years. Chen Xiao Xi is a cute and small girl with a lot of positive energy. She gets to know Jiang Chen, a tall and pro...",A Love So Beautiful,comedy romance,ts77034,0.579787,2017
9,"Love is as tough as it is sweet for a lovestruck teenager, whose relationship with her next-door neighbor transforms as they grow into adulthood.",A Love So Beautiful,comedy drama,ts357033,0.573275,2020


Then I can bias again towards recent movies

In [39]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=0.2,
    title_weight=3,
    genre_weight=1,
    recency_weight=5,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,"After his ad agency goes bankrupt, an indebted Fırat falls for a singer at a yoga retreat and joins her on a journey of self-realization.",Doom of Love,romance drama,tm1218702,0.661874,2022
1,"An ad executive and a fashion designer-blogger don't believe in love, so they place a bet to make the other fall head over heels - with unusual tactics.",Love Tactics,comedy romance,tm1152224,0.659597,2022
2,"An LA girl, unlucky in love, falls for an East Coast guy on a dating app and decides to surprise him for the holidays, only to discover that she's been catfished. This lighthearted roman...",Love Hard,romance comedy,tm1093099,0.656336,2021
3,"Fidelity tells a story of marital fidelity, in particular the one of Carlo and Margherita, a young couple who needs to face the deflagrant consequences of an alleged betrayal. Their rela...","Devotion, a Story of Love and Desire",drama romance,ts280892,0.648806,2022
4,This black humor pan-Arabic anthology series is about love in general – and relationships in particular.,"Love, Life & Everything in Between",comedy drama romance,ts329711,0.644286,2022
5,"A modern love story set in the near future where an AI building is powered by human feelings. Due to a software glitch, it falls in love with a real girl, escapes the building into the b...",AI Love You,scifi comedy romance drama,tm1160424,0.639196,2022
6,"Often (mis)guided by a cheeky imaginary wizard, an awkward and lonely 20-something struggles to get out of his own way in his quest for a girlfriend.",Eternally Confused and Eager for Love,comedy romance,ts330012,0.638304,2022
7,The story of a family and the various situations navigated by a husband and wife.,Love,thriller drama,tm946277,0.632558,2020
8,"Fashion assistant Maca has just about got her life together after a devastating breakup, when Leo, the man who broke her heart returns. Seeking support from best friends, Adriana and Jim...",Sounds Like Love,comedy music romance drama,tm876608,0.623931,2021
9,"Haruto Asakura falls in love with hairdresser Misaki Ariake and asks her out. Watching Misaki Ariake work hard to achieve what she wants, Haruto Asakura, who almost gave up his dream to ...",Love Like the Falling Petals,drama romance,tm1119015,0.617896,2022


Or maybe to older ones

In [40]:
result = app.query(
    advanced_query,
    description_query_text="Heartfelt lovely romantic comedy for a cold autumn evening.",
    title_query_text="love",
    genre_query_text="drama comedy romantic",
    description_weight=0.2,
    title_weight=3,
    genre_weight=1,
    recency_weight=-10,
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,"Two young kids fall in love with each other. But the passion is too consuming for the parents of Jade. The parents try to stop them from seeing each other. But when this doesn't work, Da...",Endless Love,romance drama,tm130586,2.646004,1981
1,"Anil, a street singer, is humiliated and driven out of Bombay along with his mother. However, he soon becomes a famous performer but the enemies from his past try to destroy his hard-ear...",Disco Dancer,drama romance,tm52815,2.575501,1982
2,"An honest man dreams of a better life for his family, but a childhood friend leads him into a world of crime that keeps happiness just out of reach.",Ujala,romance crime drama,tm358653,2.574353,1959
3,Two talented song-and-dance men team up after the war to become one of the hottest acts in show business. In time they befriend and become romantically involved with the beautiful Haynes...,White Christmas,romance comedy,tm16479,2.569005,1954
4,"Brian Cohen is an average young Jewish man, but through a series of ridiculous events, he gains a reputation as the Messiah. When he's not dodging his followers or being scolded by his s...",Life of Brian,comedy,tm70993,2.565624,1979
5,"Geeky student Arnie Cunningham falls for Christine, a rusty 1958 Plymouth Fury, and becomes obsessed with restoring the classic automobile to her former glory. As the car changes, so doe...",Christine,horror thriller european,tm54450,2.564791,1983
6,"Maharaj Brajbhan lives a wealthy lifestyle in Bharatpur, India along with his wife, Badi Rani, but have been unable to conceive for Bharatpur, and have no choice but to leave it's reigns...",Bandie,drama action romance,tm16812,2.563961,1978
7,"Two small children and a ship's cook survive a shipwreck and find safety on an idyllic tropical island. Soon, however, the cook dies and the young boy and girl are left on their own. Day...",The Blue Lagoon,romance action drama,tm98978,2.557070,1980
8,"In the 1930s, bored waitress Bonnie Parker falls in love with an ex-con named Clyde Barrow and together they start a violent crime spree through the country, stealing cars and robbing ba...",Bonnie and Clyde,crime drama action,tm119281,2.556204,1967
9,"Ragab, a poor sailor, returns home to Alexandria after three years of absence, during which he tried to save money to marry his one true love, Hamedah. But there's trouble on the harbor,...",Dark Waters,action drama romance thriller,tm204541,2.555254,1956


notice that every movie before 1984 has the same recency score as our largest period time is 40 years.

### Search with a movie and queries

Now I found a movie in the last result set that I have already seen but I would love something similar. White Christmas (1954) is a story about people forming a rock-band and of course there is some love along the way. It has the id "tm16479".

I can query my dataset using this movie, too! Let's adjust the defined query first:

In [41]:
with_movie_query = advanced_query.with_vector(movie, sl.Param("movie_id"))

with this modification, I can add a movie from the dataset to the search.

Let's first just focus on movies with similar descriptions. The first result will of course be the movie we are searching with.

In [42]:
result = app.query(
    with_movie_query,
    description_query_text="",
    title_query_text="",
    genre_query_text="",
    description_weight=1,
    title_weight=0,
    genre_weight=0,
    recency_weight=0,
    movie_id="tm16479",
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,Two talented song-and-dance men team up after the war to become one of the hottest acts in show business. In time they befriend and become romantically involved with the beautiful Haynes...,White Christmas,romance comedy,tm16479,0.742294,1954
1,"Go backstage with French rap duo Bigflo & Oli in this intimate music documentary, then join the superstar siblings as they embark on a major tour.",Bigflo & Oli: Hip Hop Frenzy,documentation music,tm937044,0.411362,2020
2,"Sing along and dance with Bailey, Franny, Kip, Lulu and Tilly in this collection of music videos that will get you in the mood to move and groove!",Word Party Songs,animation family,ts251607,0.401688,2020
3,Exploration into the tense relationship of success and romance.,Love or Money,romance,tm997617,0.398556,2021
4,"An amalgamation of four different love stories: Seenayya and Suvarna, a middle-class couple in a small town; Gautham, an uber-cool youngster romancing in the streets of Paris with his gi...",World Famous Lover,romance drama,tm848443,0.396190,2020
5,"A dream of the hope for intimacy and love in a brutal, divisive world.",Condom Lead,drama,tm1036195,0.395414,2013
6,"Aditya, Joe, Kedar and Rob form a rock band, but break up after they fail to make a success of it. They establish regular lives until they decide to reunite and take another shot at fulf...",Rock On!!,drama music,tm37945,0.393382,2008
7,The opposite worlds of two dancers in Colombia clash on and off the dance floor when their ambition to succeed leads them down a treacherous path.,Savage Rhythm,drama music,ts326029,0.391042,2022
8,"This documentary spotlights Debbie Allen's career and follows her group of dance students as they prepare for Allen's annual ""Hot Chocolate Nutcracker,"" a reimagining of the classic ballet.",Dance Dreams: Hot Chocolate Nutcracker,documentation family,tm946927,0.390560,2020
9,"Singing and dreaming together, a talented singer-songwriter and a same-aged keyboardist add harmony and love to each other's lives.",Wish You,drama romance music,tm989502,0.389622,2021


I got back movies that focus more on the rock-band aspect of the movie. With query parameters I can try to skew the results to be more related to family.

In [43]:
result = app.query(
    with_movie_query,
    description_query_text="family",
    title_query_text="",
    genre_query_text="",
    description_weight=1,
    title_weight=0,
    genre_weight=0,
    recency_weight=0,
    movie_id="tm16479",
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,Two talented song-and-dance men team up after the war to become one of the hottest acts in show business. In time they befriend and become romantically involved with the beautiful Haynes...,White Christmas,romance comedy,tm16479,0.769386,1954
1,An estranged family gathers together in New York for an event celebrating the artistic work of their father.,The Meyerowitz Stories (New and Selected),comedy drama,tm244206,0.552056,2017
2,"A new couple, their exes and their children navigate the emotional challenges and tricky logistics of blended family life in this Swedish dramedy.",Bonus Family,drama comedy reality family documentation music european,ts55939,0.538323,2017
3,"Reunited in their hometown for their father's funeral, two self-interested brothers meet a peculiar woman who shares a huge secret about their family.",The Bros,comedy fantasy,tm357708,0.535995,2017
4,A separated couple live together for their child's sake in this satirical dramedy about what it means to be a good parent and spouse in today's world.,Everything Will Be Fine,comedy drama,ts302058,0.522525,2021
5,"Years after his father disowns his adopted brother for marrying a woman of lower social standing, a young man goes on a mission to reunite his family.",Kabhi Khushi Kabhie Gham,drama romance,tm36475,0.520603,2001
6,"The loves, heartbreak, jealousy and pain of the three Armoza sisters – Luna, Rachelika, and Becky - their parents, grandparents and children, set during the early-mid 20th century in Jer...",The Beauty Queen of Jerusalem,drama,ts252119,0.519797,2021
7,A dramatic story about a girl and her giving father.,Sen Benim HerŞeyimsin,comedy,tm311456,0.518786,2016
8,Three prosperous women -- including a mother and her daughter -- fall for a seductive man in Colombia's Coffee Triangle.,Playing with Fire,drama,ts87680,0.511996,2019
9,Three grown siblings must rethink their idea of family when they learn their parents are getting divorced and have their eyes on new partners.,Love Storm,family,ts83449,0.511733,2016


Taking a closer look at the description of these result movies, we can see that they share some aspect of the movie we are searching with - blended together with some notion of family:

After all, this evening I am in the mood for something light - I can also skew the results towards a specific genre: comedy

In [44]:
result = app.query(
    with_movie_query,
    description_query_text="",
    title_query_text="",
    genre_query_text="comedy",
    description_weight=1,
    title_weight=0,
    genre_weight=2,
    recency_weight=0,
    movie_id="tm16479",
    limit=TOP_N,
)

df = sl.PandasConverter.to_pandas(result)
sl.PandasConverter.format_date_column(df, "release_timestamp", "release_year", year_only=True)

,description,title,genres,id,similarity_score,release_year
0,Two talented song-and-dance men team up after the war to become one of the hottest acts in show business. In time they befriend and become romantically involved with the beautiful Haynes...,White Christmas,romance comedy,tm16479,0.830139,1954
1,"On a bet, a gridiron hero at John Hughes High School sets out to turn a bespectacled plain Jane into a beautiful and popular prom queen in this outrageous send-up of the teen movies of t...",Not Another Teen Movie,comedy,tm58382,0.714763,2001
2,Argentine actor and comedian Fernando Sanjiao uses humor and impersonations to explore the concepts of masculinity and fatherhood in modern times.,Fernando Sanjiao: Hombre,comedy,tm416134,0.714334,2018
3,"Mo Gilligan blends smooth moves and sharp humor as he riffs on humble beginnings, family dynamics and the complex art of dancing in the club.",Mo Gilligan: Momentum,comedy,tm832844,0.713081,2019
4,"Comedies Honest, introspective comic Simon Amstell digs deep and delivers a uniquely vulnerable stand-up set on love, ego, intimacy and ayahuasca.",Simon Amstell: Set Free,comedy,tm825952,0.712658,2019
5,Get ready to scream with laughter! Sam Kinison's primal scream will echo down through the comedy hall of fame summing up so much of modernity's horror and stupidity. This concert was tap...,Sam Kinison: Family Entertainment Hour,comedy,tm8687,0.708027,1991
6,Romantic comedy following four couples who meet through a dating website. Four men go on a heroic mission to help four women and wind up experiencing a series of mishaps.,"All's Well, Ends Well",romance comedy,tm37573,0.707968,2012
7,"Ronny Chieng (""The Daily Show,"" ""Crazy Rich Asians"") takes center stage in this stand-up special and riffs on modern American life and more.",Ronny Chieng: Asian Comedian Destroys America!,comedy,tm844662,0.706744,2019
8,"With unique individual perspectives that converge into a unified voice, sketch comedy group Astronomy Club delivers a smart and absurd brand of humor.",Astronomy Club: The Sketch Show,comedy,ts223308,0.705561,2019
9,"In pursuit of both success and validation, a group of tech-savvy individuals juggle intimate encounters, first impressions and romantic opportunities.",Slay,romance comedy,tm1038025,0.705414,2021
